In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [2]:
data=pd.read_csv("/kaggle/input/insurance/insurance.csv")
data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
y=data["sex"]
x=data.iloc[:,[2,3]].values

data.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
#normalization
x=(x-np.min(x))/(np.max(x)-np.min(x))


In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3)

In [6]:
#KNN Model
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=3)


In [7]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=knn, X=x_train, y=y_train,cv=10)
#estimator knn seçilir çünkü datayı knn kadar parçaya ayırıp hepsini tahmin edecek
print("average accuracy:",np.mean(accuracies))
print("stdeviation accuracy:",np.std(accuracies))

average accuracy: 0.4741935483870967
stdeviation accuracy: 0.06107761193945486


In [8]:
knn.fit(x_train,y_train)
print("test accuracy:",knn.score(x_test,y_test))

test accuracy: 0.5074626865671642


In [9]:
#Grid Search CV
from sklearn.model_selection import GridSearchCV
grid={"n_neighbors":np.arange(1,50)}
knn=KNeighborsClassifier()
knn_cv=GridSearchCV(knn,grid, cv=10)
knn_cv.fit(x_train,y_train)
# en iyi knn değerini grid searchle buluruz.

print("tuned hyperparameter K:",knn_cv.best_params_) #en iyi parametreyi seçiyoruz.
print("best score:",knn_cv.best_score_) 

tuned hyperparameter K: {'n_neighbors': 23}
best score: 0.5116449325097231


In [10]:
#Grid search cv with Logistic Reg

from sklearn.linear_model import LogisticRegression
param_grid={"C":np.logspace(-3,3,7),"penalty":["l1","l2"]}

logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(x_train,y_train)
print("tuned hyperparameters:(best parameters):",logreg_cv.best_params_)
print("accuracy:",logreg_cv.best_score_)

ValueError: Invalid parameter n_neighbors for estimator LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [11]:

logreg2=LogisticRegression(C=1,penalty="l1")
logreg2.fit(x_train,y_train)
print("score:",logreg2.score(x_test,y_test))


ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.